<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google/meridian/blob/main/demo/Meridian_Getting_Started.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google/meridian/blob/main/demo/Meridian_Getting_Started.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

# **Introduction to Meridian Demo**

Welcome to the Meridian end-to-end demo. This simplified demo showcases the fundamental functionalities and basic usage of the library, including working examples of the major modeling steps:


<ol start="0">
  <li><a href="#install">Install</a></li>
  <li><a href="#load-data">Load the data</a></li>
  <li><a href="#configure-model">Configure the model</a></li>
  <li><a href="#model-diagnostics">Run model diagnostics</a></li>
  <li><a href="#generate-summary">Generate model results & two-page output</a></li>
  <li><a href="#generate-optimize">Run budget optimization & two-page output</a></li>
  <li><a href="#save-model">Save the model object</a></li>
</ol>


Note that this notebook skips all of the exploratory data analysis and preprocessing steps. It assumes that you have completed these tasks before reaching this point in the demo.

This notebook utilizes sample data. As a result, the numbers and results obtained might not accurately reflect what you encounter when working with a real dataset.

<a name="install"></a>
## Step 0: Install

1\. Make sure you are using one of the available GPU Colab runtimes which is **required** to run Meridian. You can change your notebook's runtime in `Runtime > Change runtime type` in the menu. All users can use the T4 GPU runtime which is sufficient to run the demo colab, free of charge. Users who have purchased one of Colab's paid plans have access to premium GPUs (such as V100, A100 or L4 Nvidia GPU).

2\. Install the latest version of Meridian, and verify that GPU is available.

In [ ]:
# Install meridian: from PyPI @ latest release
!pip install --upgrade google-meridian[colab,and-cuda]

# Install meridian: from PyPI @ specific version
# !pip install google-meridian[colab,and-cuda]==1.1.1

# Install meridian: from GitHub @HEAD
# !pip install --upgrade "google-meridian[colab,and-cuda] @ git+https://github.com/google/meridian.git@main"

In [ ]:
import arviz as az
import IPython
from meridian import constants
from meridian.analysis import analyzer
from meridian.analysis import formatter
from meridian.analysis import optimizer
from meridian.analysis import summarizer
from meridian.analysis import visualizer
from meridian.data import data_frame_input_data_builder
from meridian.data import test_utils
from meridian.model import model
from meridian.model import prior_distribution
from meridian.model import spec
import numpy as np
import pandas as pd
# check if GPU is available
from psutil import virtual_memory
import tensorflow as tf
import tensorflow_probability as tfp

ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))
print(
    'Num GPUs Available: ',
    len(tf.config.experimental.list_physical_devices('GPU')),
)
print(
    'Num CPUs Available: ',
    len(tf.config.experimental.list_physical_devices('CPU')),
)

<a name="load-data"></a>
## Step 1: Load the data

Load the [simulated dataset in CSV format](https://github.com/google/meridian/blob/main/meridian/data/simulated_data/csv/geo_all_channels.csv) as follows.

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

1\. Read the data into a Pandas DataFrame.

In [ ]:
# Update this filepath to the actual path of your file
filepath = "/content/final_combined_mmm_df_refresh_Apr15.csv"

# Read the data from the local file into a Pandas DataFrame
df = pd.read_csv(filepath)

In [ ]:
df.head()

3\. Offer the components to the builder. Note that the components may be offered all at once or piecewise.

In [ ]:
media_cols = [c for c in df.columns if "impressions" in c and "ios" in c]
spend_cols = [c for c in df.columns if "spend" in c and "ios" in c]
media_channels = [c[12:] for c in media_cols]
media_channels

In [ ]:
df.columns

2\. Create a DataFrameInputDataBuilder instance.


In [ ]:
# Rename the 'date' column to 'time' to match Meridian's expectation
df_renamed = df.rename(columns={'date': 'time'})

builder = data_frame_input_data_builder.DataFrameInputDataBuilder(
    kpi_type='revenue'
)

builder = (
    builder.with_kpi(df_renamed, kpi_col="daily_revenue", time_col="time")
)

builder = builder.with_media(
    df_renamed,
    media_cols=media_cols,
    media_spend_cols=spend_cols,
    media_channels=media_channels,
    time_col="time",
)

data = builder.build()

Note that the simulated data here does not contain reach and frequency. We recommend including reach and frequency data whenever they are available. For information about the advantages of utilizing reach and frequency, see [Bayesian Hierarchical Media Mix Model Incorporating Reach and Frequency Data](https://research.google/pubs/bayesian-hierarchical-media-mix-model-incorporating-reach-and-frequency-data/#:~:text=By%20incorporating%20R%26F%20into%20MMM,based%20on%20optimal%20frequency%20recommendations.). For code snippet for loading reach and frequency data, see [Load geo-level data with reach and frequency](https://developers.google.com/meridian/docs/user-guide/load-geo-data-with-rf)

The documentation provides guidance for instances where reach and frequency data is accessible for specific channels. Additionally, for information about how to load other data types and formats, including data with reach and frequency, see [Supported data types and formats](https://developers.google.com/meridian/docs/user-guide/supported-data-types-formats).

<a name="configure-model"></a>
## Step 2: Configure the model

Meridian uses Bayesian framework and Markov Chain Monte Carlo (MCMC) algorithms to sample from the posterior distribution.

1\. Inititalize the `Meridian` class by passing the loaded data and the customized model specification. One advantage of Meridian lies in its capacity to calibrate the model directly through ROI priors, as described in [Media Mix Model Calibration With Bayesian Priors](https://research.google/pubs/media-mix-model-calibration-with-bayesian-priors/). In this particular example, the ROI priors for all media channels are identical, with each being represented as Lognormal(0.2, 0.9).

In [ ]:
roi_mu = 0.2  # Mu for ROI prior for each media channel.
roi_sigma = 0.9  # Sigma for ROI prior for each media channel.
prior = prior_distribution.PriorDistribution(
    roi_m=tfp.distributions.LogNormal(roi_mu, roi_sigma, name=constants.ROI_M)
)

# this is mopstly full of dfault params, but can iterate from here
model_spec = spec.ModelSpec(
    prior=prior_distribution.PriorDistribution(),
    media_effects_dist='log_normal',
    hill_before_adstock=False,
    max_lag=8,
    unique_sigma_for_each_geo=False,
    media_prior_type='roi',
    roi_calibration_period=None,
    rf_prior_type='roi',
    rf_roi_calibration_period=None,
    organic_media_prior_type='contribution',
    organic_rf_prior_type='contribution',
    non_media_treatments_prior_type='contribution',
    knots=None,
    baseline_geo=None,
    holdout_id=None,
    control_population_scaling_id=None,
)
mmm = model.Meridian(input_data=data, model_spec=model_spec)

2\. Use the `sample_prior()` and `sample_posterior()` methods to obtain samples from the prior and posterior distributions of model parameters. If you are using the T4 GPU runtime this step may take about 10 minutes for the provided data set.

In [ ]:
%%time
mmm.sample_prior(500)
mmm.sample_posterior(
    n_chains=10, n_adapt=2000, n_burnin=500, n_keep=1000, seed=1
)

For more information about configuring the parameters and using a customized model specification, such as setting different ROI priors for each media channel, see [Configure the model](https://developers.google.com/meridian/docs/user-guide/configure-model).

<a name="model-diagnostics"></a>
## Step 3: Run model diagnostics

After the model is built, you must assess convergence, debug the model if needed, and then assess the model fit.

1\. Assess convergence. Run the following code to generate r-hat statistics. R-hat close to 1.0 indicate convergence. R-hat < 1.2 indicates approximate convergence and is a reasonable threshold for many problems.

In [ ]:
model_diagnostics = visualizer.ModelDiagnostics(mmm)
model_diagnostics.plot_rhat_boxplot()

In [ ]:
model_diagnostics.predictive_accuracy_table()

In [ ]:
model_diagnostics.plot_prior_and_posterior_distribution()

2\. Assess the model's fit by comparing the expected sales against the actual sales.

In [ ]:
model_fit = visualizer.ModelFit(mmm)
model_fit.plot_model_fit()

For more information and additional model diagnostics checks, see [Modeling diagnostics](https://developers.google.com/meridian/docs/user-guide/model-diagnostics).

<a name="generate-summary"></a>
## Step 4: Generate model results & two-page output

To export the two-page HTML summary output, initialize the `Summarizer` class with the model object. Then pass in the filename, filepath, start date, and end date to `output_model_results_summary` to run the summary for that time duration and save it to the specified file.

In [ ]:
mmm_summarizer = summarizer.Summarizer(mmm)

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

In [ ]:
filepath = '/content/drive/MyDrive'
start_date = '2021-01-25'
end_date = '2024-01-15'
mmm_summarizer.output_model_results_summary(
    'summary_output.html', filepath, start_date, end_date
)

Here is a preview of the two-page output based on the simulated data:

In [ ]:
IPython.display.HTML(filename='/content/drive/MyDrive/summary_output.html')

For a customized two-page report, model results summary table, and individual visualizations, see [Model results report](https://developers.google.com/meridian/docs/user-guide/generate-model-results-report) and [plot media visualizations](https://developers.google.com/meridian/docs/user-guide/plot-media-visualizations).





<a name="generate-optimize"></a>
## Step 5: Run budget optimization & generate an optimization report

You can choose what scenario to run for the budget allocation. In default scenario, you find the optimal allocation across channels for a given budget to maximize the return on investment (ROI).

1\. Instantiate the `BudgetOptimizer` class and run the `optimize()` method without any customization, to run the default library's Fixed Budget Scenario to maximize ROI.

In [ ]:
%%time
budget_optimizer = optimizer.BudgetOptimizer(mmm)
optimization_results = budget_optimizer.optimize()

In [ ]:
display(optimization_results)

2\. Export the 2-page HTML optimization report, which contains optimized spend allocations and ROI.

In [ ]:
filepath = '/content/drive/MyDrive'
optimization_results.output_optimization_summary(
    'optimization_output.html', filepath
)

In [ ]:
IPython.display.HTML(filename='/content/drive/MyDrive/optimization_output.html')

For information about customized optimization scenarios, such as flexible budget scenarios, see [Budget optimization scenarios](https://developers.google.com/meridian/docs/user-guide/budget-optimization-scenarios). For more information about optimization results summary and individual visualizations, see [optimization results output](https://developers.google.com/meridian/docs/user-guide/generate-optimization-results-output) and [optimization visualizations](https://developers.google.com/meridian/docs/user-guide/plot-optimization-visualizations).

<a name="save-model"></a>
## Step 6: Save the model object

We recommend that you save the model object for future use. This helps you to  avoid repetitive model runs and saves time and computational resources. After the model object is saved, you can load it at a later stage to continue the analysis or visualizations without having to re-run the model.


Run the following codes to save the model object:

In [ ]:
file_path = '/content/drive/MyDrive/saved_mmm.pkl'
model.save_mmm(mmm, file_path)

Run the following codes to load the saved model:

In [ ]:
mmm = model.load_mmm(file_path)

To mount your Google Drive, run the cell below and follow the authentication steps.

# Task
Generate Python code to create a waterfall chart of effect sizes from the `meridian.analysis` module using the `mmm` object.

## Explore `meridian.analysis`

### Subtask:
Inspect the `meridian.analysis` module to identify classes or functions related to visualizing model results or contributions.


**Reasoning**:
Inspect the `meridian.analysis` module to identify relevant classes or functions for visualizing model results and contributions.



In [ ]:
import inspect
import meridian.analysis

# List all attributes and methods in meridian.analysis
analysis_attributes = dir(meridian.analysis)
print("Attributes and methods in meridian.analysis:")
for attr_name in analysis_attributes:
    print(attr_name)

print("\nExamining relevant classes/functions:")
# Examine classes/functions that might be related to visualization
for attr_name in analysis_attributes:
    attr = getattr(meridian.analysis, attr_name)
    if inspect.isclass(attr) or inspect.isfunction(attr):
        if "plot" in attr_name.lower() or "visualiz" in attr_name.lower() or "results" in attr_name.lower() or "contribut" in attr_name.lower() or "summary" in attr_name.lower():
            print(f"\n--- {attr_name} ---")
            try:
                print(inspect.getdoc(attr))
            except TypeError:
                print("No docstring available.")

**Reasoning**:
Based on the previous output, the `visualizer`, `analyzer`, and `summarizer` modules seem most relevant to visualizing model results and contributions. Inspecting these modules further will help identify the specific class or function for a waterfall chart.



In [ ]:
import inspect
import meridian.analysis.analyzer
import meridian.analysis.summarizer
import meridian.analysis.visualizer

print("Examining meridian.analysis.visualizer:")
visualizer_attributes = dir(meridian.analysis.visualizer)
for attr_name in visualizer_attributes:
    attr = getattr(meridian.analysis.visualizer, attr_name)
    if inspect.isclass(attr) or inspect.isfunction(attr):
        if "waterfall" in attr_name.lower():
            print(f"\n--- {attr_name} ---")
            try:
                print(inspect.getdoc(attr))
            except TypeError:
                print("No docstring available.")

print("\nExamining meridian.analysis.analyzer:")
analyzer_attributes = dir(meridian.analysis.analyzer)
for attr_name in analyzer_attributes:
    attr = getattr(meridian.analysis.analyzer, attr_name)
    if inspect.isclass(attr) or inspect.isfunction(attr):
         if "waterfall" in attr_name.lower():
            print(f"\n--- {attr_name} ---")
            try:
                print(inspect.getdoc(attr))
            except TypeError:
                print("No docstring available.")

print("\nExamining meridian.analysis.summarizer:")
summarizer_attributes = dir(meridian.analysis.summarizer)
for attr_name in summarizer_attributes:
    attr = getattr(meridian.analysis.summarizer, attr_name)
    if inspect.isclass(attr) or inspect.isfunction(attr):
         if "waterfall" in attr_name.lower():
            print(f"\n--- {attr_name} ---")
            try:
                print(inspect.getdoc(attr))
            except TypeError:
                print("No docstring available.")

**Reasoning**:
Since directly searching for "waterfall" in the module attributes did not yield results, I will inspect all classes and functions within the `visualizer`, `analyzer`, and `summarizer` modules to find the relevant visualization function by examining their docstrings for keywords like "plot", "chart", "contribution", or "effect".



In [ ]:
import inspect

print("Examining meridian.analysis.visualizer for plotting functions:")
visualizer_attributes = dir(meridian.analysis.visualizer)
for attr_name in visualizer_attributes:
    attr = getattr(meridian.analysis.visualizer, attr_name)
    if inspect.isclass(attr) or inspect.isfunction(attr):
        try:
            docstring = inspect.getdoc(attr)
            if docstring and ("plot" in docstring.lower() or "chart" in docstring.lower() or "contribution" in docstring.lower() or "effect" in docstring.lower()):
                 print(f"\n--- {attr_name} ---")
                 print(docstring)
        except TypeError:
            pass # Ignore attributes without docstrings

print("\nExamining meridian.analysis.analyzer for plotting functions:")
analyzer_attributes = dir(meridian.analysis.analyzer)
for attr_name in analyzer_attributes:
    attr = getattr(meridian.analysis.analyzer, attr_name)
    if inspect.isclass(attr) or inspect.isfunction(attr):
        try:
            docstring = inspect.getdoc(attr)
            if docstring and ("plot" in docstring.lower() or "chart" in docstring.lower() or "contribution" in docstring.lower() or "effect" in docstring.lower()):
                 print(f"\n--- {attr_name} ---")
                 print(docstring)
        except TypeError:
            pass # Ignore attributes without docstrings

print("\nExamining meridian.analysis.summarizer for plotting functions:")
summarizer_attributes = dir(meridian.analysis.summarizer)
for attr_name in summarizer_attributes:
    attr = getattr(meridian.analysis.summarizer, attr_name)
    if inspect.isclass(attr) or inspect.isfunction(attr):
        try:
            docstring = inspect.getdoc(attr)
            if docstring and ("plot" in docstring.lower() or "chart" in docstring.lower() or "contribution" in docstring.lower() or "effect" in docstring.lower()):
                 print(f"\n--- {attr_name} ---")
                 print(docstring)
        except TypeError:
            pass # Ignore attributes without docstrings

**Reasoning**:
Based on the examination of the `meridian.analysis` modules, the `MediaEffects` class within `meridian.analysis.visualizer` appears to be the most relevant for visualizing media effects, which is consistent with generating a waterfall chart of effect sizes. I will now confirm if `MediaEffects` has a method for plotting waterfall charts by checking its attributes and docstrings.



In [ ]:
import inspect
import meridian.analysis.visualizer

print("Examining meridian.analysis.visualizer.MediaEffects for waterfall plotting method:")
media_effects_attributes = dir(meridian.analysis.visualizer.MediaEffects)
for attr_name in media_effects_attributes:
    attr = getattr(meridian.analysis.visualizer.MediaEffects, attr_name)
    if inspect.isfunction(attr):
        try:
            docstring = inspect.getdoc(attr)
            if docstring and ("waterfall" in docstring.lower() or "contribution" in docstring.lower() or "effect" in docstring.lower()):
                 print(f"\n--- {attr_name} ---")
                 print(docstring)
            elif "waterfall" in attr_name.lower():
                 print(f"\n--- {attr_name} ---")
                 print("No docstring available but name suggests relevance.")
        except TypeError:
            pass # Ignore attributes without docstrings

**Reasoning**:
Based on the previous inspection, the `visualizer` module contains the `MediaEffects` class, but none of its methods explicitly mention "waterfall" in their names or docstrings. However, the original request mentioned using `meridian.analysis.analyzer.ModelResults.plot_waterfall()`, and the traceback in the user's notebook indicates an `AttributeError: module 'meridian.analysis.analyzer' has no attribute 'ModelResults'`. It's possible the function name or class has changed in the library version being used. I will now inspect the `analyzer` module again, but this time looking for any plotting or results-related functions or classes, to see if the waterfall functionality has been moved or renamed.



In [ ]:
import inspect
import meridian.analysis.analyzer

print("Examining meridian.analysis.analyzer for plotting/results functions:")
analyzer_attributes = dir(meridian.analysis.analyzer)
for attr_name in analyzer_attributes:
    attr = getattr(meridian.analysis.analyzer, attr_name)
    if inspect.isclass(attr) or inspect.isfunction(attr):
        try:
            docstring = inspect.getdoc(attr)
            if docstring and ("plot" in docstring.lower() or "chart" in docstring.lower() or "results" in docstring.lower() or "contribution" in docstring.lower() or "effect" in docstring.lower()):
                 print(f"\n--- {attr_name} ---")
                 print(docstring)
            elif "results" in attr_name.lower() or "contribution" in attr_name.lower() or "effect" in attr_name.lower():
                 print(f"\n--- {attr_name} ---")
                 print("No docstring available but name suggests relevance.")

        except TypeError:
            pass # Ignore attributes without docstrings

## Identify alternative plotting method

### Subtask:
Based on the exploration, find a suitable method to generate a chart showing the effect sizes or contributions of different factors.


**Reasoning**:
Re-examine the relevant visualizer classes (`MediaEffects`, `MediaSummary`, `ModelFit`, `ModelDiagnostics`) and their methods to find one suitable for visualizing effect sizes or contributions, even if not explicitly called "waterfall".

